In [1]:
from brian2 import *
import numpy as np
import matplotlib.pyplot as plt

In [11]:
data_path = '/media/maria/DATA1/Documents/NeuroMatchAcademy2020_dat/unzipped_files/Richards_2017-10-31.tar'
spike_times = np.load(data_path+'/'+'spikes.times.npy') * 1000 # Unbinned spike times in ms
clusters_annotation = np.load(data_path+'/'+'clusters._phy_annotation.npy')
spike_clusters = np.load(data_path+'/'+'spikes.clusters.npy')

In [13]:
def sort_into_cells(spike_times,clusters_annotation,spike_clusters):
    spike_time_cells = np.empty(len(clusters_annotation), dtype=object) # Initalise empty object
    for i in (np.arange(len(np.unique(spike_clusters)))):
      # Create a spike time arrays, where each array in the array is a spike time of a cell
      spike_time_cells[i] = spike_times[(np.where(spike_clusters == i)[0])]
    return spike_time_cells

In [14]:
spike_time_cells=sort_into_cells(spike_times,clusters_annotation,spike_clusters)

In [17]:
print(spike_time_cells[0])

[[1.50146667e+03]
 [2.12953333e+03]
 [2.31823333e+03]
 ...
 [2.96655043e+06]
 [2.96662953e+06]
 [2.96669323e+06]]
